In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from scipy.ndimage import label, find_objects
from matplotlib.colors import ListedColormap
from PIL import Image  # Necesario para cargar la imagen PNG

# Rutas de los archivos NIfTI y la imagen PNG
#nii_path = '/home/rmartin/projects/Deep_INGENIO/OUTPUT_5/Ingenio_RYC_00005_HE [x=37575,y=28718,w=540,h=540]_type_map.nii.gz'
nii_path = '/home/rmartin/projects/Deep_INGENIO/inference_prueba/ex3/Ingenio_RYC_00032_HE [d=2,x=100580,y=51426,w=1080,h=1080]_type_map.nii.gz'
nii_img = nib.load(nii_path)
img_data = nii_img.get_fdata()

#nii_path_i = '/home/rmartin/projects/Deep_INGENIO/OUTPUT_5/Ingenio_RYC_00005_HE [x=37575,y=28718,w=540,h=540]_instance_map.nii.gz'
nii_path_i = '/home/rmartin/projects/Deep_INGENIO/inference_prueba/ex3/Ingenio_RYC_00032_HE [d=2,x=100580,y=51426,w=1080,h=1080]_instance_map.nii.gz'
nii_img_i = nib.load(nii_path_i)
img_data_i = nii_img_i.get_fdata()

# Rotar la imagen 90 grados a la derecha (270 grados a la izquierda)
rotated_img_data = np.rot90(img_data, k=3, axes=(0, 1))
rotated_img_data_i = np.rot90(img_data_i, k=3, axes=(0, 1))

# Girar la imagen en espejo
mirrored_img_data = np.fliplr(rotated_img_data)
mirrored_img_data_i = np.fliplr(rotated_img_data_i)

# Crear nuevas imágenes NIfTI con los datos procesados
new_nii_img = nib.Nifti1Image(mirrored_img_data, affine=nii_img.affine)
new_nii_img_i = nib.Nifti1Image(mirrored_img_data_i, affine=nii_img_i.affine)

# Obtener los datos de las nuevas imágenes
img_data_new = new_nii_img.get_fdata()
img_data_new_i = new_nii_img_i.get_fdata()

# Contar tipos de círculos (img_data_new)
unique_types = np.unique(img_data_new)
num_nucleos_types = len(unique_types) - 1  # Excluimos el valor 0 que representa el fondo

print("Tipos de núcleos:")
print("Número de tipos:", num_nucleos_types)

# Contar núcleos totales por tipo de círculo
nucleo_count = {}

for nucleo_type in unique_types:
    if nucleo_type == 0:
        continue  # Ignorar el fondo
    
    # Crear máscara para el tipo de círculo actual
    mask_type = (img_data_new == nucleo_type)
    
    # Encontrar los valores únicos de núcleos dentro de esta máscara
    unique_nuclei_values = np.unique(img_data_new_i[mask_type])
    
    # Excluir el valor 0 (fondo) si está presente
    if 0 in unique_nuclei_values:
        unique_nuclei_values = unique_nuclei_values[unique_nuclei_values != 0]
    
    # Contar el número total de núcleos para este tipo de círculo
    nucleo_count[nucleo_type] = len(unique_nuclei_values)

print("\nNúmero de núcleos por tipo de círculo:")
for nucleo_type, count in nucleo_count.items():
    if int(nucleo_type) == 1:
        print(f"Tipo Tumor: {count} núcleos")
    if int(nucleo_type) == 2:
        print(f"Tipo Normal: {count} núcleos")
    if int(nucleo_type) == 3:
        print(f"Tipo Stroma: {count} núcleos")

# Crear una nueva imagen solo con los contornos
contour_img = np.zeros_like(img_data_new_i)

for nucleo_type in unique_types:
    if nucleo_type == 0:
        continue  # Ignorar el fondo
    
    # Crear máscara para el tipo de círculo actual
    mask_type = (img_data_new == nucleo_type)
    
    # Etiquetar los objetos en la máscara
    labeled_array, num_features = label(mask_type)
    
    # Encontrar los objetos y sus contornos
    for obj_idx in range(1, num_features + 1):
        obj_mask = (labeled_array == obj_idx)
        
        # Obtener el slice del objeto
        slices = find_objects(labeled_array)[obj_idx - 1]
        
        # Crear un array de ceros con la forma de la máscara
        obj_contour = np.zeros_like(obj_mask, dtype=np.bool)
        
        # Iterar sobre las dimensiones y añadir los contornos
        for dim, sl in enumerate(slices):
            if sl is not None:
                obj_contour[sl] = True
        
        # Convertir a entero
        obj_contour = obj_contour.astype(int)
        
        # Agregar el contorno con el tipo de núcleo correspondiente
        contour_img += obj_contour * nucleo_type

# Colormap para los contornos
contour_cmap = ListedColormap(['none', 'red', 'green', 'blue'])

# Cargar la imagen 
png_path = '/home/rmartin/projects/Deep_INGENIO/inference_prueba/Ingenio_RYC_00032_HE [d=2,x=100580,y=51426,w=1080,h=1080].jpg'
img_png = np.array(Image.open(png_path))

# Visualización de las imágenes en subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Configuración para la imagen de Tipos de Núcleos
cmap_types = ListedColormap(['white', 'blue', 'black', 'green'])
bounds_types = [0.5, 1.5, 2.5, 3.5]
norm_types = plt.Normalize(vmin=1, vmax=3)

axes[0].imshow(img_data_new )
axes[0].set_title('Tipos de nucleos')
axes[0].axis('off')

# Configuración para la imagen de Instancias de Núcleos
axes[1].imshow(img_data_new_i)
axes[1].set_title('Instancias de nucleos')
axes[1].axis('off')

# Configuración para la imagen PNG
axes[2].imshow(img_png)
axes[2].set_title('Imagen')
axes[2].axis('off')

# Añadir texto con el número total de núcleos por tipo de círculo en el subplot de Tipos de nucleos
for nucleo_type, count in nucleo_count.items():
    if nucleo_type == 0:
        continue
    y, x = np.where(img_data_new == nucleo_type)
    text_x = np.mean(x)
    text_y = np.mean(y)
    axes[0].text(text_x, text_y, f'{int(nucleo_type)}: {count}', color='red', fontsize=10, ha='center', va='center')

plt.tight_layout()
plt.show()

print("Tipos")
print(img_data_new.shape)
#print (img_data_new[520])
print(np.unique(img_data_new))
print(len(np.unique(img_data_new))-1)

print("Instancias")
print(img_data_new_i.shape)
#print (img_data_new_i)
print(np.unique(img_data_new_i))
print(len(np.unique(img_data_new_i))-1)